In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.0.1-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [ ]:
#import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType


# Create a SparkSession
spark = SparkSession.builder.appName("SkillDrill8-1").getOrCreate()

In [ ]:
df = spark.read.csv("../Resources/zillow_housing_prices.csv", header=True, inferSchema=True) 


In [ ]:
df = df.withColumnRenamed('2020-01', 'Jan')
df = df.withColumnRenamed('2020-02', 'Feb')


In [ ]:
df.createOrReplaceTempView('houses') 

In [ ]:
df.show(10)

+---+--------+------------+------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------

In [ ]:
spark.sql("""
  select
    StateName,
    avg(Jan) 
  from houses 
  GROUP BY StateName
  """).show(10)

+------------+------------------+
|   StateName|          avg(Jan)|
+------------+------------------+
|        Utah|          345200.0|
|      Hawaii|         572868.75|
|   Minnesota|256823.59550561797|
|        Ohio| 179073.9393939394|
|      Oregon| 342466.6666666667|
|    Arkansas|162611.42857142858|
|       Texas|266331.35135135136|
|North Dakota|230916.66666666666|
|Pennsylvania|246516.66666666666|
| Connecticut|291745.28301886795|
+------------+------------------+
only showing top 10 rows



In [ ]:
spark.sql("""
  select
    StateName,
    avg(Feb) 
  from houses 
  GROUP BY StateName
  """).show(10)

+------------+------------------+
|   StateName|          avg(Feb)|
+------------+------------------+
|        Utah|  347562.962962963|
|      Hawaii|          592525.0|
|   Minnesota|261769.76744186046|
|        Ohio|182818.30065359478|
|      Oregon|348658.82352941175|
|    Arkansas|165128.57142857142|
|       Texas| 269829.6703296703|
|North Dakota|          232660.0|
|Pennsylvania| 250119.6261682243|
| Connecticut| 297518.8679245283|
+------------+------------------+
only showing top 10 rows



In [ ]:
out_df=spark.sql("""
  select
    StateName,
    avg(Feb) 
  from houses 
  GROUP BY StateName
  HAVING AVG(Feb)>100000
  """)

In [ ]:
out_df.show()
 

+--------------------+------------------+
|           StateName|          avg(Feb)|
+--------------------+------------------+
|                Utah|  347562.962962963|
|              Hawaii|          592525.0|
|           Minnesota|261769.76744186046|
|                Ohio|182818.30065359478|
|              Oregon|348658.82352941175|
|            Arkansas|165128.57142857142|
|               Texas| 269829.6703296703|
|        North Dakota|          232660.0|
|        Pennsylvania| 250119.6261682243|
|         Connecticut| 297518.8679245283|
|            Nebraska|          198640.0|
|              Nevada|          327770.0|
|          Washington| 439433.3333333333|
|            Illinois|232948.19277108434|
|            Oklahoma|165754.83870967742|
|District of Columbia|          575700.0|
|            Delaware| 285189.4736842105|
|          New Mexico|          228962.5|
|       West Virginia|          184400.0|
|            Missouri|209020.83333333334|
+--------------------+------------

In [ ]:
out_df.toPandas().to_csv('houses_out_pandas.csv') 